<a href="https://colab.research.google.com/github/v5v5/trading/blob/main/trade_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URL to download quotes EURUSD

https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true

In [418]:
import pandas as pd
from urllib.request import Request, urlopen
from io import StringIO
import datetime
import time
import random
import math

In [419]:
dt_from = datetime.date(2000,1,1)
# dt_from = datetime.date(2019,1,1)
ut_from = str(int(time.mktime(dt_from.timetuple())))
ut_from

'946684800'

In [420]:
dt_to = datetime.date.today()
ut_to = str(int(time.mktime(dt_to.timetuple())))
ut_to

'1619740800'

In [421]:
# url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1262304000&period2=1619654400&interval=1d&events=history&includeAdjustedClose=true'
url='https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=' + ut_from + '&period2=' + ut_to + '&interval=1d&events=history&includeAdjustedClose=true'

In [422]:
headers={'User-Agent': 'Mozilla/5.0'}

In [423]:
req = Request(url, headers=headers)

In [424]:
page = urlopen(req)
# page = urlopen(url)

In [425]:
page = urlopen(url)

In [426]:
content = page.read()

In [427]:
text = content.decode('UTF-8')

In [428]:
print(text[:300])

Date,Open,High,Low,Close,Adj Close,Volume
2003-12-01,1.203398,1.204007,1.194401,1.196501,1.196501,0
2003-12-02,1.196101,1.210903,1.194600,1.208897,1.208897,0
2003-12-03,1.209000,1.213003,1.207700,1.212298,1.212298,0
2003-12-04,1.212004,1.214403,1.204398,1.208094,1.208094,0
2003-12-05,1.207802,1.2190


In [429]:
data = StringIO(text)

In [430]:
df1 = pd.read_csv(data, sep=',', header='infer')

In [431]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume
4538,2021-04-22,1.203818,1.207001,1.200394,1.203775,1.203775,0.0
4539,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0.0
4540,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0.0
4541,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0.0
4542,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0.0
4543,2021-04-29,1.213578,1.215214,1.210419,1.213592,1.213592,0.0
4544,2021-04-30,1.212562,1.213298,1.211974,1.212121,1.212121,0.0


In [432]:
df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')

In [433]:
df1['DayOfWeek'] = df1['Date'].dt.dayofweek

In [434]:
df1.tail(7)

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
4538,2021-04-22,1.203818,1.207001,1.200394,1.203775,1.203775,0.0,3
4539,2021-04-23,1.201793,1.207799,1.201389,1.201475,1.201475,0.0,4
4540,2021-04-26,1.208956,1.211827,1.206374,1.208985,1.208985,0.0,0
4541,2021-04-27,1.207921,1.209190,1.205793,1.208167,1.208167,0.0,1
4542,2021-04-28,1.208854,1.210214,1.205764,1.209031,1.209031,0.0,2
4543,2021-04-29,1.213578,1.215214,1.210419,1.213592,1.213592,0.0,3
4544,2021-04-30,1.212562,1.213298,1.211974,1.212121,1.212121,0.0,4


In [435]:
df2 = df1.drop(['High','Low', 'Adj Close', 'Volume'], axis=1)

In [436]:
df2.tail(7)

,Date,Open,Close,DayOfWeek
4538,2021-04-22,1.203818,1.203775,3
4539,2021-04-23,1.201793,1.201475,4
4540,2021-04-26,1.208956,1.208985,0
4541,2021-04-27,1.207921,1.208167,1
4542,2021-04-28,1.208854,1.209031,2
4543,2021-04-29,1.213578,1.213592,3
4544,2021-04-30,1.212562,1.212121,4


In [437]:
df2['Close-1'] = df2['Close'].shift(1)
df2['Open-1'] = df2['Open'].shift(1)

In [438]:
df2.tail(7)

,Date,Open,Close,DayOfWeek,Close-1,Open-1
4538,2021-04-22,1.203818,1.203775,3,1.203239,1.203254
4539,2021-04-23,1.201793,1.201475,4,1.203775,1.203818
4540,2021-04-26,1.208956,1.208985,0,1.201475,1.201793
4541,2021-04-27,1.207921,1.208167,1,1.208985,1.208956
4542,2021-04-28,1.208854,1.209031,2,1.208167,1.207921
4543,2021-04-29,1.213578,1.213592,3,1.209031,1.208854
4544,2021-04-30,1.212562,1.212121,4,1.213592,1.213578


In [439]:
df2['%Close'] = (df2['Close'] - df2['Close-1']) / df2['Close-1'] * 100
df2['%Open'] = (df2['Open'] - df2['Open-1']) / df2['Open-1'] * 100

In [440]:
df2.head(7)

,Date,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open
0,2003-12-01,1.203398,1.196501,0,NaN,NaN,NaN,NaN
1,2003-12-02,1.196101,1.208897,1,1.196501,1.203398,1.036021,-0.606366
2,2003-12-03,1.209000,1.212298,2,1.208897,1.196101,0.281331,1.078421
3,2003-12-04,1.212004,1.208094,3,1.212298,1.209000,-0.346779,0.248470
4,2003-12-05,1.207802,1.218695,4,1.208094,1.212004,0.877498,-0.346699
5,2003-12-08,1.216797,1.222001,0,1.218695,1.207802,0.271274,0.744741
6,2003-12-09,1.222105,1.224995,1,1.222001,1.216797,0.245008,0.436227


In [441]:
def take_decision(index):
  row = df2.loc[index]
  return random.choice([-row['Open'], +row['Open']])

In [442]:
def is_guessed(index):
  row = df2.loc[index]
  if (index < 1):
    return True
  return (math.copysign(1, row['%Open']) == math.copysign(1, row['decision']))

In [443]:
def make_deal(index):
  k = df2.loc[index, 'k'] 
  decision = df2.loc[index, 'decision'] 
  return decision * (2 ** k)

In [444]:
def discover_trend(index):
  if (index < 1):
    df2.loc[index, 'trend'] = 0
    return
  open_curr = df2.loc[index, '%Open']
  open_prev = df2.loc[index-1, '%Open']
  if (math.copysign(1, open_prev) == math.copysign(1, open_curr)):
    df2.loc[index, 'trend'] = df2.loc[index-1, 'trend'] + 1
  else:
    df2.loc[index, 'trend'] = 0

In [445]:
def trade():
  for index, row in df2.iterrows():
    df2.loc[index, 'decision'] = take_decision(index)
    if (is_guessed(index)):
      k = 0
    else:
      k += 1
    df2.loc[index, 'k'] = k
    deal = make_deal(index)
    df2.loc[index, 'deal'] = deal
    discover_trend(index)

In [446]:
trade()
df2

,Date,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open,decision,k,deal,trend
0,2003-12-01,1.203398,1.196501,0,NaN,NaN,NaN,NaN,-1.203398,0.0,-1.203398,0.0
1,2003-12-02,1.196101,1.208897,1,1.196501,1.203398,1.036021,-0.606366,-1.196101,0.0,-1.196101,0.0
2,2003-12-03,1.209000,1.212298,2,1.208897,1.196101,0.281331,1.078421,-1.209000,1.0,-2.418000,0.0
3,2003-12-04,1.212004,1.208094,3,1.212298,1.209000,-0.346779,0.248470,-1.212004,2.0,-4.848016,1.0
4,2003-12-05,1.207802,1.218695,4,1.208094,1.212004,0.877498,-0.346699,-1.207802,0.0,-1.207802,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4540,2021-04-26,1.208956,1.208985,0,1.201475,1.201793,0.625065,0.596026,-1.208956,1.0,-2.417912,0.0
4541,2021-04-27,1.207921,1.208167,1,1.208985,1.208956,-0.067660,-0.085611,-1.207921,0.0,-1.207921,0.0
4542,2021-04-28,1.208854,1.209031,2,1.208167,1.207921,0.071513,0.077240,1.208854,0.0,1.208854,0.0
4543,2021-04-29,1.213578,1.213592,3,1.209031,1.208854,0.377244,0.390783,-1.213578,1.0,-2.427156,1.0


In [447]:
df2.describe()

,Open,Close,DayOfWeek,Close-1,Open-1,%Close,%Open,decision,k,deal,trend
count,4496.000000,4496.000000,4545.000000,4495.000000,4495.000000,4478.000000,4478.000000,4496.000000,4545.000000,4496.000000,4545.000000
mean,1.259347,1.259373,2.000000,1.259383,1.259357,0.004146,0.004358,-0.005863,0.962816,0.000224,0.989659
std,0.119864,0.119853,1.414369,0.119865,0.119875,0.746274,0.741814,1.265164,1.395945,67.394139,1.716769
min,1.039393,1.039047,0.000000,1.039047,1.039393,-13.352634,-13.322116,-1.598184,0.000000,-2386.446336,0.000000
25%,1.159017,1.158997,1.000000,1.158950,1.158991,-0.328796,-0.327189,-1.259930,0.000000,-2.119768,0.000000
50%,1.257561,1.258004,2.000000,1.258004,1.257561,0.006124,0.002507,-1.055058,0.000000,-1.057189,0.000000
75%,1.344203,1.344023,3.000000,1.344050,1.344213,0.337267,0.338316,1.254595,1.000000,1.494623,1.000000
max,1.598184,1.598798,4.000000,1.598798,1.598184,17.307961,16.607811,1.591495,11.000000,2270.584832,24.000000
